In [7]:
import pymysql
import time


In [8]:
class MysqlSearch(object):

    def __init__(self):
        self.get_conn()
        self.today=time.strftime("%m%d", time.localtime())
    
    def get_conn(self):
        """ 获取连接 """
        try:
            self.conn = pymysql.connect(
                host='localhost',
                port=3306,
                user='birth',
                passwd='birth',
                db='birth',
                charset='utf8')
        except Exception as e:
            print('Error:%s' %e)

    def close_conn(self):
        try:
            # 关闭连接
            if self.conn:
                self.conn.close()
        except Exception as e:
            print('Error: %s' %e)
    
    def get_person(self):
        sql = 'SELECT * FROM user WHERE birth="%s"'%(self.today)
        cursor = self.conn.cursor()
        cursor.execute(sql)
        data = cursor.fetchall()
        data = [dict(zip([k[0] for k in cursor.description], row)) for row in data]
        cursor.close()
        self.close_conn()
        return data


In [9]:
sql=MysqlSearch()
lucky_person=sql.get_person()
# print(lucky_person)

In [10]:
import smtplib
from email.mime.text import MIMEText
from email.header import Header
 
# 第三方 SMTP 服务
mail_host="smtp.exmail.qq.com"  #设置服务器
mail_user="####@###.com"    #用户名
mail_pass="#######"   #口令 
 
 
sender = '###@###.com'
receivers = ['###@###.com']  # 接收邮箱

mail_msg='''
<table border="2">
  <tr>
    <th>uid</th>
    <th>姓名</th>
    <th>备注</th>
  </tr>
'''

for i in lucky_person:
    mail_msg+='''
      <tr>
        <td>%s</td>
        <td>%s</td>
        <td>%s</td>
      </tr>
    '''%(i['uid'],i['name'],i['ps'])
mail_msg+='</table>'
 
message = MIMEText(mail_msg, 'html', 'utf-8')
message['From'] = Header("birthReminder", 'utf-8')
message['To'] =  Header("余生", 'utf-8')
 
subject = '你的这些好友今天生日...'
message['Subject'] = Header(subject, 'utf-8')
 
 
try:
    smtpObj = smtplib.SMTP_SSL(host=mail_host) 
    smtpObj.connect(mail_host, 465)    # 465 为 SMTP 端口号
    
    smtpObj.login(mail_user,mail_pass)

    smtpObj.sendmail(sender, receivers, message.as_string())
    
    print ("邮件发送成功")
except smtplib.SMTPException:
    print ("Error: 无法发送邮件")

邮件发送成功
